In [55]:
import gdown
import zipfile
import pandas as pd
import os
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import plotly.express as px

tqdm.pandas(desc="Processing")
pd.set_option('display.max_columns', None)

In [2]:
tmp_folder = "./energy_dataset"
os.makedirs(tmp_folder, exist_ok=True)

# ---------------------------
# Step 1: Download Core + Bonus ZIPs
# ---------------------------
zip_files = {
    "core": "https://drive.google.com/uc?id=13o_2ojFRCCqwmYMN3w3qu5fQxieXATTd",
    "bonus": "https://drive.google.com/uc?id=1Hvqi5nv66m3b1aEN23NnUOBkVKQrfP5z"
}

extracted_csv_paths = []

for name, url in zip_files.items():
    zip_path = os.path.join(tmp_folder, f"{name}_dataset.zip")
    if os.path.exists(zip_path):
        print('Data already downloaded, skipping download')
        continue
    print(f"\nDownloading {name} ZIP...")
    gdown.download(url, zip_path, quiet=False)
    
    print(f"Extracting CSVs from {name} ZIP...")
    with zipfile.ZipFile(zip_path, "r") as z:
        for member in z.namelist():
            if member.endswith(".csv") and "__MACOSX" not in member:
                print(f"  Extracting {member}")
                z.extract(member, tmp_folder)
                extracted_csv_paths.append(os.path.join(tmp_folder, member))

# ---------------------------
# Step 2: Print list of CSV files
# ---------------------------
print("\nAll extracted CSV files:")
if not extracted_csv_paths:
    extracted_csv_paths = list(Path(tmp_folder).glob('**/*.csv'))
for csv_path in extracted_csv_paths:
    print(f" - {os.path.basename(csv_path)}")

# ---------------------------
# Step 3: Load CSVs into Pandas
# ---------------------------
pdf_dict = {}
for csv_path in extracted_csv_paths:
    csv_name = os.path.basename(csv_path)
    print(f"\nLoading {csv_name} into Pandas...")
    pdf_dict[csv_name] = pd.read_csv(csv_path, encoding="latin1")
    print(f"  {csv_name} loaded, shape: {pdf_dict[csv_name].shape}")

Data already downloaded, skipping download
Data already downloaded, skipping download

All extracted CSV files:
 - meter-readings-aug-2025.csv
 - meter-readings-dec-2025.csv
 - meter-readings-july-2025.csv
 - meter-readings-june-2025.csv
 - meter-readings-may-2025.csv
 - meter-readings-nov-2025.csv
 - meter-readings-oct-2025.csv
 - meter-readings-sept-2025.csv
 - building_metadata.csv
 - meter-readings-april-2025.csv
 - meter-readings-feb-2025.csv
 - meter-readings-jan-2025.csv
 - meter-readings-march-2025.csv
 - weather_data_hourly_2025.csv

Loading meter-readings-aug-2025.csv into Pandas...
  meter-readings-aug-2025.csv loaded, shape: (760368, 25)

Loading meter-readings-dec-2025.csv into Pandas...
  meter-readings-dec-2025.csv loaded, shape: (766320, 25)

Loading meter-readings-july-2025.csv into Pandas...
  meter-readings-july-2025.csv loaded, shape: (760368, 25)

Loading meter-readings-june-2025.csv into Pandas...
  meter-readings-june-2025.csv loaded, shape: (735840, 25)

Loading

In [89]:
def calculate_eui_no_double_counting(df_meter, df_building):
    # --- STEP 1: CLEAN IDS & TYPES ---
    # Convert IDs to string and strip decimals to ensure matching (e.g., 311.0 -> "311")
    df_meter['simscode_clean'] = df_meter['simscode'].astype(str).str.replace(r'\.0$', '', regex=True)
    df_building['buildingnumber_clean'] = df_building['buildingnumber'].astype(str).str.replace(r'\.0$', '', regex=True)
    
    # Ensure gross area is numeric
    df_building['grossarea'] = pd.to_numeric(df_building['grossarea'], errors='coerce')

    # --- STEP 2: HANDLE DOUBLE COUNTING (HEAT vs. STEAM) ---
    # 1. Identify all buildings that have a STEAM meter
    steam_buildings = set(df_meter[df_meter['utility'] == 'STEAM']['simscode_clean'])
    
    # 2. Create a filter mask
    # Logic: Keep the row IF:
    #   (Utility is NOT 'HEAT')  OR  (Building does NOT have Steam)
    # This removes 'HEAT' rows only for buildings that also have 'STEAM'
    mask_keep = (df_meter['utility'] != 'HEAT') | (~df_meter['simscode_clean'].isin(steam_buildings))
    
    # 3. Apply the filter
    df_clean_meter = df_meter[mask_keep].copy()
    
    # Calculate how many rows were dropped for verification
    rows_dropped = len(df_meter) - len(df_clean_meter)
    print(f"Dropped {rows_dropped} 'HEAT' rows to prevent double-counting.")

    # --- STEP 3: CONVERT TO KBTU ---
    # 1 kWh = 3.412 kBTU
    # 1 kg Steam ~ 2.632 kBTU (Enthalpy estimate)
    conversion_factors = {
        'ELECTRICITY': 3.412,
        'HEAT': 3.412,
        'GAS': 3.412,
        'COOLING': 3.412,
        'OIL28SEC': 3.412,
        'STEAM': 2.632 
    }

    # Filter for valid energy types only (removes POWER/RATE)
    df_energy = df_clean_meter[df_clean_meter['utility'].isin(conversion_factors.keys())].copy()
    
    # Apply conversion
    df_energy['factor'] = df_energy['utility'].map(conversion_factors)
    df_energy['kbtu'] = df_energy['readingvalue'] * df_energy['factor']

    # --- STEP 4: AGGREGATE & CALCULATE EUI ---
    # Sum annual energy by building
    df_total = df_energy.groupby('simscode_clean')['kbtu'].sum().reset_index()

    # Join with Building Info
    df_final = pd.merge(
        df_total,
        df_building[['buildingnumber_clean', 'buildingname', 'grossarea', 'constructiondate']],
        left_on='simscode_clean',
        right_on='buildingnumber_clean',
        how='inner'
    )

    # Calculate EUI (kBTU / sq ft)
    # Filter out zero-area buildings to avoid division by zero
    df_final = df_final[df_final['grossarea'] > 0]
    df_final['EUI'] = df_final['kbtu'] / df_final['grossarea']

    return df_final.sort_values('EUI', ascending=False)

# Usage
# df_results = calculate_eui_no_double_counting(meter_readings, building_data)
# print(df_results.head(10))

def calculate_monthly_eui(df_meter, df_building):
    # --- STEP 1: PREPARE DATA & DATES ---
    # Ensure date column is datetime objects
    df_meter['readingtime'] = pd.to_datetime(df_meter['readingtime'])
    
    # Extract Month and Year for grouping
    df_meter['month'] = df_meter['readingtime'].dt.month
    df_meter['year'] = df_meter['readingtime'].dt.year
    
    # Clean IDs for joining
    df_meter['simscode_clean'] = df_meter['simscode'].astype(str).str.replace(r'\.0$', '', regex=True)
    df_building['buildingnumber_clean'] = df_building['buildingnumber'].astype(str).str.replace(r'\.0$', '', regex=True)
    df_building['grossarea'] = pd.to_numeric(df_building['grossarea'], errors='coerce')

    # --- STEP 2: HANDLE DOUBLE COUNTING (Same logic as before) ---
    # Identify buildings with STEAM and remove their HEAT readings
    steam_buildings = set(df_meter[df_meter['utility'] == 'STEAM']['simscode_clean'])
    mask_keep = (df_meter['utility'] != 'HEAT') | (~df_meter['simscode_clean'].isin(steam_buildings))
    df_clean_meter = df_meter[mask_keep].copy()

    # --- STEP 3: CONVERT TO KBTU ---
    conversion_factors = {
        'ELECTRICITY': 3.412,
        'HEAT': 3.412,
        'GAS': 3.412,
        'COOLING': 3.412,
        'OIL28SEC': 3.412,
        'STEAM': 2.632 
    }
    
    df_energy = df_clean_meter[df_clean_meter['utility'].isin(conversion_factors.keys())].copy()
    df_energy['factor'] = df_energy['utility'].map(conversion_factors)
    df_energy['kbtu'] = df_energy['readingvalue'] * df_energy['factor']

    # --- STEP 4: AGGREGATE BY MONTH ---
    # Group by Building AND Month/Year
    df_monthly = df_energy.groupby(['simscode_clean', 'year', 'month'])['kbtu'].sum().reset_index()

    # --- STEP 5: JOIN & CALCULATE MONTHLY EUI ---
    df_final = pd.merge(
        df_monthly,
        df_building[['buildingnumber_clean', 'buildingname', 'grossarea']],
        left_on='simscode_clean',
        right_on='buildingnumber_clean',
        how='inner'
    )

    # Calculate Monthly EUI
    df_final = df_final[df_final['grossarea'] > 0]
    df_final['Monthly_EUI'] = df_final['kbtu'] / df_final['grossarea']

    # Sort by time for plotting
    return df_final.sort_values(['buildingname', 'year', 'month'])

In [26]:
pdf_dict.keys()

dict_keys(['meter-readings-aug-2025.csv', 'meter-readings-dec-2025.csv', 'meter-readings-july-2025.csv', 'meter-readings-june-2025.csv', 'meter-readings-may-2025.csv', 'meter-readings-nov-2025.csv', 'meter-readings-oct-2025.csv', 'meter-readings-sept-2025.csv', 'building_metadata.csv', 'meter-readings-april-2025.csv', 'meter-readings-feb-2025.csv', 'meter-readings-jan-2025.csv', 'meter-readings-march-2025.csv', 'weather_data_hourly_2025.csv'])

In [27]:
mega_table = pd.concat([ls for n, ls in pdf_dict.items() if 'meter' in n])

In [31]:
mega_table.to_csv('mega_table.csv', index=False)

In [33]:
new_mega = mega_table.dropna()

In [34]:
new_mega.to_csv('mega_table_dropped.csv', index=False)

In [39]:
building_data = pdf_dict['building_metadata.csv']

In [40]:
building_data.columns

Index(['buildingnumber', 'buildingname', 'campusname', 'address', 'city',
       'state', 'postalcode', 'county', 'formalname', 'alsoknownas',
       'grossarea', 'floorsaboveground', 'floorsbelowground',
       'constructiondate', 'latitude', 'longitude'],
      dtype='str')

In [73]:
power_usage = calculate_eui_no_double_counting(new_mega, building_data)

Dropped 121936 'HEAT' rows to prevent double-counting.


In [90]:
power_usage_monthly = calculate_monthly_eui(new_mega, building_data)

In [74]:
power_usage2 = power_usage.dropna()

In [92]:
power_usage_monthly2 = power_usage_monthly.dropna()

In [93]:
power_usage_monthly2 = power_usage_monthly2.reset_index(drop=True)

In [94]:
power_usage_monthly2.to_csv('energy_EUI_monthly.csv')

In [83]:
power_usage2.to_csv('test.csv')

In [85]:
power_usage2 = power_usage2.reset_index(drop=True)

In [87]:
power_usage_clean = power_usage2.drop([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
power_usage_clean.to_csv('energy_EUI_cleaned.csv', index=False)

In [53]:
power_usage3 = power_usage2.drop([120, 76])

In [54]:
power_usage3.to_csv('./energy_useage.csv', index=False)

In [66]:
power_usage3.columns

Index(['simscode_clean', 'kbtu', 'buildingnumber_clean', 'buildingname',
       'grossarea', 'constructiondate', 'formalname', 'EUI'],
      dtype='str')

In [60]:
power_usage4 = power_usage3.drop([220, 129, 130])

In [ ]:
fig = px.scatter(x=power_usage2['constructiondate'], y=power_usage2['EUI'])

In [76]:
display(fig)